To run the code you should download the notebook locally to execute the program.
You need to install Selenium and chromedrive to be able to scrape data from the website.

https://selenium-python.readthedocs.io/
\n
https://chromedriver.chromium.org/downloads

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

ModuleNotFoundError: No module named 'selenium'

In [ ]:
from time import sleep

In [ ]:
# open browser by webdrive
driver =  webdriver.Chrome()

# name of shop
shop = 'genz_official'
url = 'https://shopee.vn/'+shop+'?page=0&sortBy=sales'
print(url)
driver.get(url)
sleep(3)

In [ ]:
# get product information on each page
def get_product_one_page():
    # load page source into the program
    page_source = BeautifulSoup(driver.page_source)
    
# only get products in the "best seller" box
    parents_tags = page_source.find_all('div', class_ = 'shop-search-result-view')

    
    for result in parents_tags: 
        name_result = result.find_all('div', class_ = 'PFM7lj')
        price_current_result = result.find_all('span', class_ = '_29R_un')
        solds_result = result.find_all('div', class_ = 'go5yPW')

    list_product_names = []
    list_product_price_current = []
    list_product_price_original = []
    list_product_solds = []
    
    for item in range(len(name_result)):
#         extract text in div . tag
        name_item = name_result[item].text
        price_current_item = price_current_result[item].text
        solds_item = solds_result[item].text
        
        list_product_names.append(name_item)
        list_product_price_current.append(price_current_item)
        list_product_solds.append(solds_item)
        item = item+1
    
    df = pd.DataFrame()
    df['name'] = list_product_names
    df['current_price'] = list_product_price_current
    df['solds'] = list_product_solds

    return df

In [ ]:
# get product information on all pages
def get_product_all_pages(number_of_page):
    all_page = pd.DataFrame()
    number_of_pages = number_of_page
    for page in range(number_of_pages):
        one_page = get_product_one_page()
        all_page = all_page.append(one_page)
        sleep(2)
#         scroll the page down to the "next" button
        driver.execute_script('window.scrollTo(X=0,Y=4350);')
        next_button = driver.find_element_by_xpath('//*[@id="main"]/div/div[3]/div/div[2]/div/div[3]/div[3]/div[2]/div[3]/button[4]')
        driver.execute_script("arguments[0].click();", next_button)        
    sleep(3)
    return all_page
        

In [ ]:
data = pd.DataFrame()

In [ ]:
data = get_product_all_pages(2)

In [ ]:
df1 = data

In [ ]:
data

In [ ]:
# replace the letter 'k' in the string to '00'
for index,row in df1.iterrows():
    row['solds'] = row['solds'].replace("k","00")

# replace the letter ',' in the string to ''
for index,row in df1.iterrows():
    row['solds'] = row['solds'].replace(",","")

In [ ]:
data

In [ ]:
# get the number in str for example: 'đã bán 37/month' => 37
df1['solds'] = df1['solds'].str.extract('(\d+)')

In [ ]:
df1 = df1.fillna(0)

In [ ]:
df1

In [ ]:
# type of current_price
type(df1.iloc[3,1])

In [ ]:
# type of solds
type(df1.iloc[3,2])

In [ ]:
# remove the dot in the string to convert to int
for index,row in df1.iterrows():
    row['current_price'] = row['current_price'].replace(".","")


In [ ]:
# df1['current_price']=current_price

In [ ]:
df1.head()

In [ ]:
# calculate revenue per product
df1['revenue_per_product'] = df1['current_price'].astype(int)*df1['solds'].astype(int)

In [ ]:
df1.head()

In [ ]:
df1.to_csv('revenue per product i')

In [ ]:
# shop's revenue
df1['revenue_per_product'].sum()